# 2D Image Segmentation of MRI scans

Steps:
1. Get dataset
1. Preprocess 2D images
    1. do a
    1. do b
1. Split dataset into train/val(/test) set
1. Setup a model
1. Set training and model parameters
1. Train model
1. Validate result


## Get dataset


In [3]:
!pip3 install nibabel

    100% |################################| 3.3MB 500kB/s ta 0:00:011


In [12]:
import matplotlib.pyplot as plt
import argparse
from glob import glob
import numpy as np
import os
import sys
from pathlib import Path
from PIL import Image
import nibabel as nib

In [7]:
5 % 3

2

In [13]:
def normalise_zero_one(image):
    """Image normalisation. Normalises image to fit [0, 1] range."""

    image = image.astype(np.float32)

    minimum = np.min(image)
    maximum = np.max(image)

    if maximum > minimum:
        ret = (image - minimum) / (maximum - minimum)
    else:
        ret = image * 0.
    return ret

def getPILMode(nii_img):
    datatype = nii_img.get_data_dtype()
    if datatype == 'uint16':
        res = 'I'
    elif datatype == 'float32':
        res = 'F'
    else:
        res = 'F'
        
    return res

def split_filename(filepath):
    path = os.path.dirname(filepath)
    filename = os.path.basename(filepath)
    base, ext = os.path.splitext(filename)
    if ext == '.gz':
        base, ext2 = os.path.splitext(base)
        ext = ext2 + ext
    return path, base, ext


def convert(img_dir, out_dir, axis=2, pct_range = (0.2,0.8), recursive= False):    
    pattern ='**/*.nii*'
    if recursive:
        fns = Path(img_dir).glob(pattern)  
    else:
        fns = glob(os.path.join(img_dir, pattern))
        
    m = 0
    for fn in fns:
        m += 1
        
        if (m % 100 == 0):
            print(m)
        
        fn = str(fn)
        path, base, _ = split_filename(fn)
#         print(fn)
        nii = nib.load(fn)
        img = nii.get_data()
        start = int(pct_range[0] * img.shape[axis])
        end = int(pct_range[1] * img.shape[axis]) + 1
        out_path = path.replace(img_dir, out_dir)
#         print(f'From {path}')
#         print(f'To {out_path}')
#         print([img.min(), img.max()])

        img = (normalise_zero_one(img)*255).astype(np.uint8)
#         print(img.dtype)            
#         print([img.min(), img.max()])
        
        os.makedirs(out_path, exist_ok=True)    
        for i in range(start, end):
            nii_slice = img[i,:,:] if axis == 0 else \
                        img[:,i,:]if axis == 1 else \
                        img[:,:,i]
             
            nii_slice = np.rot90(nii_slice)    
            I = Image.fromarray(nii_slice)#, mode=mode)
            
#             a = plt.figure()
#             plt.imshow(I)

            I.save(os.path.join(out_path, f'{base}_{i}.tiff'))
    

In [14]:
convert('data','data_2d',axis = 2, recursive=True)

100
200
300
400
500


In [55]:
# !python med2image -i data/tmp/0/pre/T1.nii.gz -d test/ -o image.jpg --sliceToConvert -1

python: can't open file 'med2image': [Errno 2] No such file or directory


In [98]:
img_dir = 'data/Amsterdam_GE3T'
pattern = '**/*.nii*'
fns =  Path(img_dir).glob('**/*.nii*')  
# fns = glob(os.path.join(img_dir, pattern))

for x in fns:
    print(x)
    


data/Amsterdam_GE3T/100/wmh.nii.gz
data/Amsterdam_GE3T/100/orig/3DT1.nii.gz
data/Amsterdam_GE3T/100/orig/3DT1_mask.nii.gz
data/Amsterdam_GE3T/100/orig/FLAIR.nii.gz
data/Amsterdam_GE3T/100/orig/FLAIR_mask.nii.gz
data/Amsterdam_GE3T/100/orig/T1.nii.gz
data/Amsterdam_GE3T/100/orig/T1_mask.nii.gz
data/Amsterdam_GE3T/100/pre/3DT1.nii.gz
data/Amsterdam_GE3T/100/pre/FLAIR.nii.gz
data/Amsterdam_GE3T/100/pre/T1.nii.gz
data/Amsterdam_GE3T/101/wmh.nii.gz
data/Amsterdam_GE3T/101/orig/3DT1.nii.gz
data/Amsterdam_GE3T/101/orig/3DT1_mask.nii.gz
data/Amsterdam_GE3T/101/orig/FLAIR.nii.gz
data/Amsterdam_GE3T/101/orig/FLAIR_mask.nii.gz
data/Amsterdam_GE3T/101/orig/T1.nii.gz
data/Amsterdam_GE3T/101/orig/T1_mask.nii.gz
data/Amsterdam_GE3T/101/pre/3DT1.nii.gz
data/Amsterdam_GE3T/101/pre/FLAIR.nii.gz
data/Amsterdam_GE3T/101/pre/T1.nii.gz
data/Amsterdam_GE3T/102/wmh.nii.gz
data/Amsterdam_GE3T/102/orig/3DT1.nii.gz
data/Amsterdam_GE3T/102/orig/3DT1_mask.nii.gz
data/Amsterdam_GE3T/102/orig/FLAIR.nii.gz
data/Amste